In [1]:
import pandas as pd
import Levenshtein

In [2]:
words = pd.read_csv('words.csv')

In [3]:
words.head()

,Français,Hiragana,Kanji
0,enseignant,きょうし,教室
1,étudiant,がくせい,学生
2,employé d'une compagnie,かいしゃいん,会社員
3,employé (avec nom de compagnie),しゃいん,社員
4,employé de banque,ぎんこういん,銀行員


In [4]:
shuffled_words = words.sample(frac=1).reset_index(drop=True)

In [5]:
def is_similar_japanese_answer(user_answer, correct_answer):
    similarity_threshold = 0.8
    distance = Levenshtein.distance(user_answer.lower(), correct_answer.lower())
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    return similarity >= similarity_threshold

In [6]:
def is_similar_french_answer(user_answer, correct_answer):
    similarity_threshold = 0.5
    if pd.isna(user_answer) or pd.isna(correct_answer):
        return False
    user_answer = user_answer.lower()
    correct_answer = correct_answer.lower()
    substring_match = user_answer in correct_answer or correct_answer in user_answer
    if substring_match:
        return True
    distance = Levenshtein.distance(user_answer, correct_answer)
    similarity = 1 - (distance / max(len(user_answer), len(correct_answer)))
    
    return similarity >= similarity_threshold

In [7]:
shuffled_sample = shuffled_words.sample(n=20)
shuffled_sample

,Français,Hiragana,Kanji
157,fièvre,ねつ,熱
61,emprunter,かります,借ります
7,descendre [à l'hôtel],とまります,泊まります
32,mettre en marche / allumer,つけます,NaN
138,dentiste,はいしゃ,歯医者
95,avoir soif,のどがかわきます,NaN
45,fermer,しめます,閉めます
69,hôpital,びょういん,病院
158,travailler,はたらきます,働きます
128,automne,あき,秋


In [10]:
seen_fr = []
correct_count = 0
score_fr = 0
shuffled_sample_jp = shuffled_sample.sample(frac=1).reset_index(drop=True)

for i in range(len(shuffled_sample_jp)):
    if shuffled_sample_jp['Français'].iloc[i] not in seen_fr:
        ask_h = input(f"Comment dire {shuffled_sample_jp['Français'].iloc[i]} en hiragana ?")
        seen_fr.append(shuffled_sample_jp['Français'].iloc[i])

        if is_similar_japanese_answer(ask_h, shuffled_sample_jp['Hiragana'].iloc[i]):
            kanji_value = shuffled_sample_jp['Kanji'].iloc[i]
            if pd.notna(kanji_value):
                ask_k = input("Et en kanji ?")
                if is_similar_japanese_answer(ask_k, kanji_value):
                    correct_count += 1
                    score_fr += 1
                    print('Correct !')
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
                else:
                    print(f"Incorrect ! La bonne réponse est : {kanji_value}")
                    print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
            else:
                correct_count += 1
                score_fr += 1
                print('Correct !')
                print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')
        else:
            print(f"Incorrect ! La bonne réponse est : {shuffled_sample_jp['Hiragana'].iloc[i]}")
            print(f'Bonnes réponses : {score_fr}/{len(shuffled_sample_jp)}')

if correct_count == len(shuffled_sample_jp):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Comment dire travailler en hiragana ?はたらきます
Et en kanji ?働きます
Correct !
Bonnes réponses : 1/20
Comment dire automne en hiragana ?あき
Et en kanji ?秋
Correct !
Bonnes réponses : 2/20
Comment dire problème en hiragana ?もんだい
Et en kanji ?問題
Correct !
Bonnes réponses : 3/20
Comment dire beaux-arts en hiragana ?びじゅ
Incorrect ! La bonne réponse est : びじゅつ
Bonnes réponses : 3/20
Comment dire hôpital en hiragana ?びょういん
Et en kanji ?病院
Correct !
Bonnes réponses : 4/20
Comment dire bibliothèque en hiragana ?としょかん
Et en kanji ?図書館
Correct !
Bonnes réponses : 5/20
Comment dire à côté / voisin en hiragana ?となり
Et en kanji ?隣
Correct !
Bonnes réponses : 6/20
Comment dire fermer en hiragana ?しめます
Et en kanji ?閉めます
Correct !
Bonnes réponses : 7/20
Comment dire employé d'une compagnie en hiragana ?かいしゃいん
Et en kanji ?会社員
Correct !
Bonnes réponses : 8/20
Comment dire apprendre (par quelqu'un) en hiragana ?ならいます
Et en kanji ?習います
Correct !
Bonnes réponses : 9/20
Comment dire bureau en hiragana ?じむしょ
Et en 

In [11]:
seen_k=[]
correct_count = 0
score_jp = 0
shuffled_sample_fr = shuffled_sample.sample(frac=1).reset_index(drop=True)
for i in range(len(shuffled_sample_fr)):
    kanji = shuffled_sample_fr['Kanji'].iloc[i]
    if pd.notna(kanji) and kanji not in seen_k:
        ask_h = input(f"Que veut dire ce kanji : {kanji}?")
        if is_similar_french_answer(ask_h, shuffled_sample_fr['Français'].iloc[i]):
            seen_k.append(kanji)
            correct_count += 1
            score_jp += 1
            print(f"Correct ! C'était bien {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')
        else:
            print(f"Incorrecte ! La bonne réponse était : {shuffled_sample_fr['Français'].iloc[i]}")
            print(f'Bonnes réponses : {score_jp}/{len(shuffled_sample_fr)}')

if correct_count == len(shuffled_sample_fr):
    print("Félicitations ! Vous avez répondu correctement à toutes les réponses.")

Que veut dire ce kanji : 図書館?cantine
Incorrecte ! La bonne réponse était : bibliothèque
Bonnes réponses : 0/20
Que veut dire ce kanji : 会社員?employé d'une entreprise
Correct ! C'était bien employé d'une compagnie
Bonnes réponses : 1/20
Que veut dire ce kanji : 食堂?cantine
Correct ! C'était bien restaurant / cantine
Bonnes réponses : 2/20
Que veut dire ce kanji : 隣?à côté
Correct ! C'était bien à côté / voisin
Bonnes réponses : 3/20
Que veut dire ce kanji : 乗り場?arrêt de bus
Correct ! C'était bien arrêt de bus/taxi
Bonnes réponses : 4/20
Que veut dire ce kanji : 泊まります?descendre
Correct ! C'était bien descendre [à l'hôtel]
Bonnes réponses : 5/20
Que veut dire ce kanji : 働きます?travailler
Correct ! C'était bien travailler
Bonnes réponses : 6/20
Que veut dire ce kanji : 問題?problème
Correct ! C'était bien problème
Bonnes réponses : 7/20
Que veut dire ce kanji : 秋?automne
Correct ! C'était bien automne
Bonnes réponses : 8/20
Que veut dire ce kanji : 歯医者?dentiste
Correct ! C'était bien dentiste
Bo